<a href="https://colab.research.google.com/github/lugsantistebanji/WCS-IA/blob/main/WCS_IA_Quetes_NLP_Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge - Tweetons !


Entraîne-toi à classifier des tweets pour dire s'ils sont positifs ou négatifs. Ca pourrait être un outil très utile pour optimiser le travail d'un community manager.

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

---

1. __Importe [cet ensemble](https://github.com/murpi/wilddata/raw/master/quests/tweets.zip) de données de tweets dans un DataFrame.__

In [2]:
link = "https://github.com/murpi/wilddata/raw/master/quests/tweets.zip"
df = pd.read_csv(link, compression='zip')

In [3]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


---

2. __Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral).__

In [6]:
df.sentiment.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [7]:
df_filter = df.loc[df.sentiment.isin(['negative', 'positive'])]

In [26]:
df_filter.shape

(16363, 4)

---

3. __Quel est le pourcentage de tweets positifs/négatifs ?__

In [11]:
df_filter.groupby('sentiment').sentiment.value_counts().apply(lambda x: 100*x/len(df_filter))

,count
sentiment,
negative,47.552405
positive,52.447595


---

4. __Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.__

In [54]:
X = df_filter['text']
y = df_filter['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=32, train_size=0.75)

In [27]:
X_train.shape

(12272,)

In [55]:
X_test.shape

(4091,)

---

5. __Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.__

In [56]:
vectorizer = CountVectorizer()
X_train_CV = vectorizer.fit_transform(X_train)
X_test_CV = vectorizer.transform(X_test)

In [30]:
X_test_CV.shape

(4091, 15806)

---

6. __Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et 0.877 pour l'ensemble de test.__

In [32]:
modelLoR = LogisticRegression()

In [35]:
modelLoR.fit(X_train_CV, y_train)

y_train_predict = modelLoR.predict(X_train_CV)
y_test_predict = modelLoR.predict(X_test_CV)

In [40]:
accuracy_train = accuracy_score(y_train, y_train_predict)
accuracy_test = accuracy_score(y_test, y_test_predict)

In [41]:
print(f"Score for train set {accuracy_train}")
print(f"Score for test set {accuracy_test}")

Score for train set 0.9659387222946545
Score for test set 0.8772916157418724


---

7. __Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?__

In [62]:
results = pd.DataFrame(X_test.copy())
results['real_sentiment'] = y_test.values
results['predicted_sentiment'] = y_test_predict

In [63]:
results.loc[results['real_sentiment'] != results['predicted_sentiment']]

,text,real_sentiment,predicted_sentiment
2670,HollowbabesHere comes the utter shite #bgt <I ...,negative,positive
18731,SUFFICATION NO BREATHING. It`s okay. There`ll...,negative,positive
12054,i wanna vote for Miley Cyrus for the mtv movie...,positive,negative
21823,I love music so much that i`ve gone through pa...,negative,positive
18464,"I can only message those who message me, if we...",positive,negative
...,...,...,...
24016,"morning world, is raining 2day so revision don...",positive,negative
7082,"the $60,000 i spent going to college was well ...",positive,negative
22042,back to work.. with no distractions..,positive,negative
18027,likes the smell of the roast in the oven mmmmm...,positive,negative


---